# Travail final: Analyse d'un sous corpus portant sur les jeux olympique de 1924 

Dans ce notebook, nous étudions de manière transversale l'enssemble du sous corpus Camile extrait.

## Imports

In [1]:
# === IMPORTS ===

import os
import pandas as pd
import numpy as np

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Traitement du langage
import spacy
from spacy.lang.fr.stop_words import STOP_WORDS
from collections import Counter

# NLP avancé
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE

# Word embeddings
from gensim.models import Word2Vec

# Nettoyage
import re
import string

# Affichage
sns.set(style="whitegrid")


## Chargement et exploration du sous corpus

In [8]:
# --- Chargement du corpus CAMille (Jeux Olympiques 1924) ---

import pandas as pd
from pathlib import Path

# Chemin du fichier (à adapter si différent)
data_path = Path("data/camille_Jeux_olympiques.xlsx")

# Chargement du fichier Excel
df = pd.read_excel(data_path)

# Aperçu des premières lignes
df.head()


,ID,JOURNAL,DATE,ANNÉE,MOIS,JOUR,JDLS,ÉDITION,PAGE,LANGUE,TEXTE
0,KB_JB838_1924-07-11_01-00005,JB838,1924-07-11,1924,7,11,5,1,5,fr-BE,"Seul 1e Marathon des Flandres n’aura pas lieu,..."
1,KB_JB838_1924-06-30_01-00001,JB838,1924-06-30,1924,6,30,1,1,1,fr-BE,"Paris est en pleines Olympiades, ce qui signif..."
2,KB_JB838_1924-04-26_01-00005,JB838,1924-04-26,1924,4,26,6,1,5,fr-BE,LA PORTÉE DES <kw>JEUX</kw> <kw>OLYMPIQUES</kw...
3,KB_JB838_1924-08-07_01-00007,JB838,1924-08-07,1924,8,7,4,1,7,fr-BE,Après lès Jeta Oiynpiqaéi UN PCU DS «TATISTIQU...
4,KB_JB838_1924-08-07_01-00005,JB838,1924-08-07,1924,8,7,4,1,5,fr-BE,Pour vos disponibilités un placement rémunérat...


In [10]:
# Ajouter une colonne 'nb_mots' basée sur le texte
df["nb_mots"] = df["TEXTE"].str.split().str.len()

# Longueur moyenne
longueur_moy = df["nb_mots"].mean()

print(f"Longueur moyenne des articles : {longueur_moy:.2f} mots")


Longueur moyenne des articles : 507.15 mots


In [ ]:
# Affichage de quelques texte bruts
# Exemple de 3 textes bruts
for i, txt in enumerate(df["TEXTE"].head(3)):
    print(f"\n--- TEXTE {i+1} ---\n")
    print(txt)



--- TEXTE 1 ---

Seul 1e Marathon des Flandres n’aura pas lieu, à. cause dos <kw>Jeux</kw> <kw>Olympiques</kw>. Celui-ci sera disputé fin du mois d'août. Le programme de la fête athlétique comportera le3 épreuves suivantes: ; , 1. Critérium international de vitesse: a) 100 m. plat; b) 150 m. plat; c) 200 m. plat, par addition de points'obtenus dans chacune de ce3 épreuves. La participation do Cockuyt et JBrochart est déjà certaine.--Des’pourparlers sont engagés aveo deux -excellents sprintera étrangers. 2. Challenge du Kilomètre. ;— Classement par trois hommes dû mémo cercle. 3. Challenge des relais. — a) Bêlais suédois (400, : 300, 200 ot 100 ui.);'. b) Bêlais olympique (1,600, 600, 400. 200 m.); c) 800 m. relaie (4xzoo m.). I 4. 5,000 mètres scratch, classement spécial par > trois hommes, pour clubs de déuxième catégorie, ~ -jdoté d'un eliallcngc. 5. 200 mètres, scratch, pour deuxième catégorie. 6 .Challenge des concours: a) Lancement du disque; : b) Lancement du javelot; 'c) Lancem

# Nétoyage et préparation du texte

In [12]:
# --- Nettoyage et préparation du texte ---

import pandas as pd
import spacy
import string
import re

# Charger le modèle spaCy français (plus complet)
nlp = spacy.load("fr_core_news_md")

# colonne TEXTE de notre DataFrame
texts = df["TEXTE"].astype(str)

# passage en minuscules
df["clean_text"] = df["TEXTE"].str.lower()


In [13]:
#Suppression ponctuation & chiffres
df["clean_text"] = df["clean_text"].apply(
    lambda x: re.sub(r"[0-9]+", " ", x)
)

df["clean_text"] = df["clean_text"].apply(
    lambda x: x.translate(str.maketrans("", "", string.punctuation))
)


In [14]:
# Suppression des stopwords
stopwords = nlp.Defaults.stop_words

def remove_stopwords(text):
    return " ".join([t for t in text.split() if t not in stopwords])

df["clean_text"] = df["clean_text"].apply(remove_stopwords)


## Tokenisation + Lemmatization

In [15]:
def tokenize(text):
    doc = nlp(text)
    return [token.text for token in doc]

df["tokens"] = df["clean_text"].apply(tokenize)


In [16]:
def lemmatize(text):
    doc = nlp(text)
    return [token.lemma_ for token in doc if not token.is_punct and not token.is_space]

df["lemmas"] = df["clean_text"].apply(lemmatize)


## Récupération de tous les articles de la page d'accueil



In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

articles = []

root_url = f"https://www.rtbf.be"
response = requests.get(root_url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
for link in soup.find_all("a", {"class":"stretched-link outline-hidden"}):
    title = link.text.strip()
    url = root_url + link.get('href')
    articles.append([url, title])

In [ ]:
# Affichage du nombre d'articles récupérés
len(articles)

In [ ]:
# Affichage des 10 premières entrées
articles[:10]

## Création d'un dataframe avec les liens et les titres des articles


In [ ]:
df = pd.DataFrame(articles, columns=['link', 'title'])
df

In [ ]:
# Sauvegarde du dataframe dans un fichier csv
df.to_csv(f"../data/rtbf_{time.strftime('%Y%m%d')}.csv", index=False)

## Téléchargement d'un article et affichage du texte

In [ ]:
# Récupération du premier lien du dataframe
article_url = df.iloc[0][0]

# Ouverture de la page
response = requests.get(article_url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
# Parsing du contenu
title = soup.find("h1").get_text(separator=' ')

paragraphs = []
for paragraph in soup.find_all("p", attrs={"class": None}):
    paragraphs.append(paragraph.get_text(separator=' ').strip())

content = "\n".join(paragraphs)
print(title)
print("==================================")
print(content)

### Nettoyage du texte à l'aide d'expressions régulières

In [ ]:
# Suppression de tout ce qui se trouve entre parenthèses
clean_content = re.sub("\([^\)]+\)", " ", content)
# Suppression des espaces multiples
clean_content = re.sub("\s+", " ", clean_content)

print(content)
print("==================================")
print(clean_content)


### Création d'un fichier avec le contenu de l'article


In [ ]:
with open("../data/rtbf_example.txt", "w") as writer:
    writer.write(f"{title}\n\n{clean_content}")

## Pour en savoir plus

- Le web scraping avec Python: https://realpython.com/beautiful-soup-web-scraper-python/
- Tutoriel sur les expressions régulières: https://www.w3schools.com/python/python_regex.asp